#

# Representing Quantum States and Operators with NumPy

## Quantum States and Operators in truncated Hilbert Spaces

### Examples

#### Pauli Operators

In [1]:
#| label: pauli-operators
import numpy as np

sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, 1j], [-1j, 0]])
sz = np.array([[1, 0], [0, -1]])

#### Spin S Systems

In [2]:
#| label: spin-operators
s = 1
d = int(2 * s + 1)

# Vector of the diagonal elements for the SZ operator
vecm = np.flip(np.arange(-s, s + 1)) #arange creates arrays ordered that starts from '-s' 
                                     #and adding '+1' arrives to 's+1'. 
                                     #Conventionally we want to have the negative numbers 
                                     #in the last place. So we need to 'flip' it.
SZ = np.diag(vecm, 0)

# Vector for SM operator (ladder down)
vec2 = vecm[:d-1]  # remove the last entry
vec3 = np.sqrt(s * (s + 1) - vec2 * (vec2 - 1))
SM = np.diag(vec3, k=-1)  # place vec3 in lower diagonal

# Construct SX and SY
SX = (SM + SM.T) / 2
SY = 1j * (SM.T - SM) / 2

#### Harmonic Oscillator

In [3]:
#| label: destroy-and-fock
def destroy(d):
    # creates a vector of the d-1 off-diagonal elements
    v=np.sqrt( np.arange(d-1) )
    # matrix with the elements of vec placed in the upper diagonal
    a=np.diag(v,k=1)
    return a

# Define the fock states
def fock(d, i):
    res = np.zeros(d)
    res[i] = 1
    return res

d = 7
zero_state = fock(d, 0)
one_state = fock(d, 1)
two_state = fock(d, 2)
three_state = fock(d, 3)

destroy_operator = destroy(d)
destroy_operator

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.41421356, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.73205081,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        2.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 2.23606798],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

## Action of the Destroy Operator on a Fock State

In [4]:
#| label: destroy-apply
# Apply the destroy operator on the one state
result_state = np.dot(destroy_operator, three_state)

print("Resulting State:")
result_state

Resulting State:

array([0.        , 0.        , 1.41421356, 0.        , 0.        ,
       0.        , 0.        ])

------------------------------------------------------------------------

> **Exercise!**
>
> Write a function `expect(O, psi)` that calculates the expectation
> value of an operator $\hat{O}$ with a given state $\psi$. Then
> calculate $\langle 2 \vert \hat{a} \vert 2 \rangle$ and
> $\langle 2 \vert \hat{a}^\dagger \hat{a} \vert 2 \rangle$.

In [5]:
# Write your code here...


In [6]:
#| code-fold: true
#| code-summary: Solution
def expect(O, psi):
    return np.dot(np.conj(psi), np.dot(O, psi))

print("Expectation Value 1:", expect(destroy_operator, fock(d, 2)))
print("Expectation Value 2:", expect(destroy_operator.T.conj().dot(destroy_operator), fock(d, 2)))

Expectation Value 1: 0.0
Expectation Value 2: 1.0

------------------------------------------------------------------------

## Partial Trace

In [7]:
#| label: ptrace
def ptrace(psi, subspace_to_keep, dim_subspace):
    dim1, dim2 = dim_subspace

    rho = np.outer(psi, psi.conj())

    # Reshape rho to separate the subsystems' degrees of freedom
    rho_reshaped = rho.reshape(dim1, dim2, dim1, dim2)

    if subspace_to_keep == 1:
        # Perform the trace over the second subsystem
        traced_out = np.trace(rho_reshaped, axis1=1, axis2=3)
    elif subspace_to_keep == 2:
        # Perform the trace over the first subsystem
        traced_out = np.trace(rho_reshaped, axis1=0, axis2=2)
    else:
        raise ValueError("subspace_to_keep must be either 1 or 2.")

    return traced_out

# Bell state between two qubits
phi_plus = ( np.kron(fock(2, 1), fock(2, 1)) + np.kron(fock(2, 0), fock(2, 0)) ) / np.sqrt(2)

# Reduced density matrix of the first qubit
rho_1 = ptrace(phi_plus, 1, (2, 2))
rho_1

array([[0.5, 0. ],
       [0. , 0.5]])